In [1]:
# To know the current file path

%pwd

'e:\\End to End Medical Chatbot\\Medical-Chatbot\\research'

In [2]:
# Make sure that we are in the root directory

import os
os.chdir("../")

%pwd

'e:\\End to End Medical Chatbot\\Medical-Chatbot'

In [4]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from huggingface_hub import hf_hub_download

c:\Users\Pritha\anaconda3\envs\medibot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Extract the Data from The PDF file

def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob = "*.pdf",
        loader_cls = PyPDFLoader
    )

    documents = loader.load()

    return documents

In [9]:
extracted_data = load_pdf_file(data = 'Data/')

In [10]:
#extracted_data

In [12]:
# Split the Data into Text Chunks

def text_spliter(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap = 20
    )

    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [13]:
text_chunks = text_spliter(extracted_data)
print(f"Length of the Text Chunks: {len(text_chunks)}")

Length of the Text Chunks: 39994


In [ ]:
# text_chunks

In [14]:
# Download the Embeddings from Hugging Face

def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(
        model_name = "sentence-transformers/all-MiniLM-L6-v2"
    )

    return embeddings

In [15]:
embeddings = download_hugging_face_embedding()

C:\Users\Pritha\AppData\Local\Temp\ipykernel_13112\1833081923.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [16]:
query_result = embeddings.embed_query("Hello World")
print(f"Length: {len(query_result)}")

Length: 384


In [ ]:
# query_result

In [17]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [1]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

# Create a Pinecone client instance
pc = Pinecone(api_key = PINECONE_API_KEY)

index_name = "aimedibot"

# Create an index
pc.create_index(
    name = index_name,
    dimension = 384,  # Replace with your model's dimensions
    metric = "cosine",  # Replace with your preferred metric

    spec = ServerlessSpec(
        cloud = "aws",
        region = "us-east-1"
    )
)

c:\Users\Pritha\anaconda3\envs\medibot\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


NameError: name 'Pinecone' is not defined

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [19]:
# Embed each chunk and upsert the embeddings into your pinecone index

from langchain_pinecone import PineconeVectorStore

docSearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding = embeddings
)

NameError: name 'index_name' is not defined

In [ ]:
# Connect to the existing index for document search

docSearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,  # Use the created index
    embedding = embeddings  # Embedding model used for vectorizing queries/documents
)

In [ ]:
docSearch

In [ ]:
retriver = docSearch.as_retriever(  # Convert the Pinecone vector store into a retriever
    search_type = "similarity",       # Use similarity-based search to find the closest matches
    search_kwargs = {'k': 3}          # Retrieve the top 3 most similar results
)

In [ ]:
retrived_docs = retriver.invoke("What is breast cancer?")

In [ ]:
retrived_docs

[Document(id='96207d27-e8cc-4304-8541-07ea498bcf94', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 671.0, 'page_label': '642', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\The Gale Encyclopedia of Medicine (3rd Edition).pdf', 'total_pages': 4505.0}, page_content='Definition\nBreast cancer is caused by the development of\nmalignant cells in the breast. The malignant cells\noriginate in the lining of the milk glands or ducts of\nthe breast (ductal epithelium), defining this malig-\nnancy as a cancer. Cancer cells are characterized by\nuncontrolled division leading to abnormal growth\nand the ability of these cells to invade normal tissue\nlocally or to spread throughout the body, in a process\ncalled metastasis.\nDescription'),
 Document(id='b314c338-6164-44e3-92e2-28ac3db45a5d', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI


# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash", 
    google_api_key = GOOGLE_API_KEY
)

In [ ]:
# Test the model
response = llm.invoke("Hello! What is Gemini?")
print(response)

content="Gemini is a family of **large language models (LLMs)** developed by Google AI. It's designed to be multimodal, meaning it can understand and generate text, images, audio, video, and code. Think of it as a very advanced AI that can process and create a wide range of content.\n\nHere's a breakdown of key aspects:\n\n*   **Multimodal Capabilities:** This is a core feature. Gemini isn't just about text like many other LLMs. It's designed to seamlessly integrate and reason across different types of information. For example, it can look at an image and write a description, or listen to audio and summarize the content.\n\n*   **Large Language Model (LLM):** Like other LLMs (such as GPT-4), Gemini is trained on a massive dataset of text and other data. This allows it to learn patterns, relationships, and nuances in language and other modalities, enabling it to generate human-quality content, translate languages, answer questions, and much more.\n\n*   **Different Versions:** Google ha

In [ ]:
# Import necessary modules from LangChain
from langchain.chains import create_tagging_chain  # For tagging and categorization tasks
from langchain.chains.combine_documents import create_stuff_documents_chain  # Combines retrieved documents
from langchain_core.prompts import ChatPromptTemplate  # Helps in structuring prompts
from langchain.chains import create_retrieval_chain  # Creates a retrieval-based chain



# Define the system prompt for the assistant
system_prompt = (
    "You are an assistant for question-answering tasks."  # Specifies the AI's role
    "Use the following pieces of retrieved context to answer the question."  # Instructs AI to use provided context
    "If you don't know the answer, say that sorry, you don't know."  # Ensures AI doesn't hallucinate answers
    "Use three sentences maximum and keep the answer concise."  # Encourages short and precise responses
    "\n\n"  # Adds a newline for better formatting
    "{context}"  # Placeholder where the retrieved context will be inserted
)



# Create a structured prompt template using ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),  # The system message containing AI instructions
        ("human", "{input}"),  # Placeholder for the user input/question
    ]
)

In [ ]:
# Creates a chain that takes the LLM (language model) and the prompt
# This chain structures the retrieved documents into a clear, usable format for the LLM to generate an answer
question_answer_chain = create_stuff_documents_chain(llm, prompt)


# Creates a Retrieval-Augmented Generation (RAG) chain
# This combines the retriever (which fetches relevant documents) with the question-answer chain
# It ensures the LLM gets the right context before generating a response
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

In [ ]:
# Sends a query to the RAG (Retrieval-Augmented Generation) chain for answering
response = rag_chain.invoke({"input": "What is Acne?"})

# Extracts and prints the "answer" field from the response dictionary
print(response["answer"])

# If the question is relevant to the data the retriever finds, 
# the model provides an accurate answer.

Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. Acne vulgaris is the medical term for common acne.


In [ ]:
# Sends a user question ("What is geography?") into the RAG (Retrieval-Augmented Generation) chain
response = rag_chain.invoke({"input": "What is geography?"})

# Extracts and prints the "answer" field from the response dictionary
print(response["answer"])

# If the retrieved context doesn't include relevant information about 
# "geography," the model responds with a polite fallback message

The provided text discusses wilderness medicine, pathologists, and skin biopsies. It does not contain any information about geography. Therefore, I don't know the answer.
